In [4]:
import torch as th
import sys
import os
import certifi
import matplotlib.pyplot as plt
from lpe.methods import MHIS
from lpe.method_utils import *
from lpe.utils import Transformer
from lpe.utils import datasets as lpe_datasets
from lpe.utils.model_adapter import ModelAdapter
from lpe.da_mhis import DA_MHIS
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Set device
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

In [ ]:
# Set model name
model_name = 'openai-community/gpt2' # or 'gelu-1l' or whatever

In [ ]:
# Run if toy model
model = Transformer.from_pretrained(model_name).to(device)

In [ ]:
# Run if HF model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
hf_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model = ModelAdapter(hf_model, tokenizer, device=device)

In [ ]:
dist_name = 'camel'
behavior = distribution_registry[dist_name](model.tokenizer, device=model.device)
orig_dists = behavior.input_dists(n_reps=N_REPS_DICT[dist_name])

In [ ]:
target = 123  # some token id
temp = 1.0    # tune as needed
ests = DA_MHIS(model, orig_dists, target, temp=temp, n_smaples=2**12, burn_in=2**9, show_progress=True)
print(ests)